In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

In [4]:
parts_path = Path(r'C:\Users\a.bagherian\Desktop\working\Diabetics Combined 18_95')

In [5]:
destination_path = Path(r'C:\Users\a.bagherian\Desktop\working\one_df_days')

## Bulding one_day_df

In [4]:
# cols = ['member', 'service', 'prescription', 'days']

In [5]:
# df = pd.DataFrame(columns=cols)
# df

,days,part


In [6]:
# df = pd.DataFrame(columns=['days', 'part'])
# df

# for i, pickle_name in enumerate(os.listdir(parts_path)):
#     part_number = i + 1
#     pickle_path = parts_path / pickle_name
    
#     # Importing
#     part = pd.read_pickle(pickle_path)

#     # # Filtering for ab rows
#     # part = part.loc[part['ab']]

#     # # Filtering for columns
#     # part = part[cols]

#     # computing member_days
#     member_days = pd.DataFrame(part.groupby('member')['days'].sum())
#     member_days['part'] = part_number

#     df = pd.concat(
#         [df, member_days],
#         axis=0
#     )

In [7]:
member_days_path = destination_path / 'member_days_18_95.pickle'
# df.to_pickle(member_days_path)

## _

In [ ]:
member_days = pd.read_pickle(member_days_path)
member_days

In [9]:
# This file contains the days of each ab service for all diabetics. so the members of
# dm that didn't receive ab aren't here.
one_df_days_path = destination_path / 'one_df_days_18_95.pickle'
one_df_days = pd.read_pickle(one_df_days_path)

In [ ]:
one_df_days

In [13]:
# Bye prescription
one_df_days['days'].describe()

count                      9565821
mean     7 days 23:58:43.716211951
std      9 days 00:20:43.045788499
min         0 days 00:01:09.120000
25%         4 days 23:59:16.800000
50%         6 days 15:59:02.400000
75%               10 days 00:00:00
max             7347 days 00:00:00
Name: days, dtype: object

In [15]:
# By member
member_days.describe() / 3

,days
count,568060.666667
mean,14 days 23:12:05.921632455
std,23 days 18:00:57.557665302
min,0 days 00:00:00
25%,2 days 08:00:00
50%,8 days 07:59:39.840000
75%,18 days 21:19:58.080000
max,2851 days 15:19:52.320000


In [20]:
member_days.loc[member_days['days'] > pd.Timedelta(0, 'day')].describe()

,days
count,1408300
mean,54 days 08:00:29.992287381
std,75 days 00:56:55.528132586
min,0 days 00:04:45.120000
25%,15 days 00:00:00
50%,33 days 07:55:12
75%,66 days 07:59:48.480000
max,8554 days 21:59:36.959999994


## Looking for outliers

In [45]:
one_df_days['days'].quantile(0.99)

Timedelta('30 days 00:00:00')

In [42]:
one_df_days.loc[one_df_days['days'] > one_df_days['days'].quantile(0.99)].describe()

,days
count,80010
mean,58 days 08:26:08.683410574
std,61 days 07:05:30.170056675
min,30 days 00:00:51.839999999
25%,40 days 00:00:00
50%,50 days 00:00:00
75%,60 days 00:00:00
max,7347 days 00:00:00


In [ ]:
limit = pd.Timedelta(30, 'day')
print(limit)
outliers = one_df_days.loc[one_df_days['days'] > limit]
outliers

In [ ]:
outliers['part'] = member_days.loc[outliers['member'], 'part'].values
outliers

In [76]:
# Saving outlier prescriptions
# outliers.to_pickle(r'C:\Users\a.bagherian\Desktop\working\one_df_days\outlier_ab_prescriptions.pkl')

## Checking if the reason of outliers are ddds

In [ ]:
ab_service = pd.read_csv(r'C:\Users\a.bagherian\Desktop\working\ab_service_final.csv',
                         index_col=0)
ab_service

In [65]:
temp = ab_service.loc[outliers['service'].unique()]
out_serv = temp.loc[temp['recheck'] == 1].index

In [67]:
out_serv

Int64Index([240, 968, 5648], dtype='int64')

In [ ]:
outliers.loc[outliers['service'].isin(out_serv)]

## _

## Removing outlier ab prescriptions and computing the member_days again

In [83]:
# df = pd.DataFrame(columns=['days', 'part'])
# df

# for i, pickle_name in enumerate(os.listdir(parts_path)):
#     part_number = i + 1
#     pickle_path = parts_path / pickle_name
    
#     # Importing
#     part = pd.read_pickle(pickle_path)

#     # Removing outliers
#     part = part.drop(outliers.loc[outliers['part'] == part_number].index, axis=0)

#     # # Filtering for ab rows
#     # part = part.loc[part['ab']]

#     # # Filtering for columns
#     # part = part[cols]

#     # computing member_days
#     member_days = pd.DataFrame(part.groupby('member')['days'].sum())
#     member_days['part'] = part_number

#     df = pd.concat(
#         [df, member_days],
#         axis=0
#     )

In [8]:
member_days_outliers_removed_path = destination_path / 'member_days_18_95_outliers_removed.pickle'
# df.to_pickle(member_days_outliers_removed_path)

In [ ]:
days_final = pd.read_pickle(member_days_outliers_removed_path)
days_final

In [11]:
days_final.loc[days_final['days'] > pd.Timedelta(0, 'day')].describe() / 3

,days
count,468589.666667
mean,17 days 00:52:59.594508685
std,21 days 04:53:26.884135489
min,0 days 00:01:35.040000
25%,5 days 00:00:00
50%,10 days 17:59:02.400000
75%,21 days 05:19:03.360000
max,1853 days 07:42:08.640000


In [ ]:
num_days = (days_final['days'] / pd.Timedelta(1, 'day')) / 3
num_days 

In [130]:
days_final.loc[num_days == 0, 'cat'] = 0
days_final.loc[(0 < num_days) & (num_days <= 5), 'cat'] = 1
days_final.loc[(5 < num_days) & (num_days <= 11), 'cat'] = 2
days_final.loc[(11 < num_days) & (num_days <= 21), 'cat'] = 3
days_final.loc[(21 < num_days), 'cat'] = 4

In [131]:
days_final['cat'].value_counts(normalize=True, dropna=False)

1.0    0.210216
2.0    0.209374
4.0    0.208825
3.0    0.196480
0.0    0.175106
Name: cat, dtype: float64

In [ ]:
days_final

In [133]:
# Saving
# The member_cat.pickle file contains categories based on ab with outliers removed. and is 18_95. The index is member.
# days_final.to_pickle(destination_path / 'member_cat.pkl')